In [ ]:
# %pip install graphviz

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import os
from xgboost import XGBClassifier

from sklearn.model_selection import RandomizedSearchCV, train_test_split

%matplotlib inline

In [ ]:
data = pd.read_csv('full_data.csv')

## Looking at the raw data:

In [ ]:
print('Size of data: {}'.format(data.shape))
print('Number of events: {}'.format(data.shape[0]))
print('Number of columns: {}'.format(data.shape[1]))

print ('\nList of features in dataset:')
for col in data.columns:
    print(col)

In [ ]:
# headings = data.columns.to_list()
# print(len(headings))
# headings_without_label = headings.pop(19)
# print(len(headings))
# headings.append('label')
# print(headings)

In [ ]:
data.drop(columns=data.columns[0], axis=1,  inplace=True)

In [ ]:
data.head()

In [ ]:
# data.to_csv('full_data.csv')

In [ ]:
print('Number of signal events: {}'.format(len(data[data.label == 1])))
print('Number of background events: {}'.format(len(data[data.label == 0])))
print('Fraction signal: {}'.format(len(data[data.label == 1])/(float)(len(data[data.label == 1]) + len(data[data.label == 0]))))

## Data Observation

In [ ]:
data_high_glu = data[data.reco_g_1 > 0.7]
data_high_glu = data_high_glu[data_high_glu.reco_g_2 > 0.7]

In [ ]:
# plt.figure()
# plt.hist(data_high_glu.jj_pt_2[data.label == 1],bins=np.linspace(0, 100, 100),
#          histtype='step',color='midnightblue',label='signal')
# plt.hist(data_high_glu.jj_pt_2[data.label == 0],bins=np.linspace(0, 100, 100),
#          histtype='step',color='firebrick',label='background')

# plt.xlabel('feature',fontsize=12)
# plt.ylabel('Events',fontsize=12)
# plt.legend(frameon=False)

In [ ]:
# signal, sig_bins = np.histogram(data_high_glu.jet_m_2[data.label == 1],bins=np.linspace(0, 100, 100))
# bkgrnd, back_bins = np.histogram(data_high_glu.jet_m_2[data.label == 0],bins=np.linspace(0, 100, 100))

# sig_sf = 10 * 7.38400e-05
# back_sf = (10 * 363) / 1.333318333

# plt.hist(sig_bins[:-1], sig_bins, weights=sig_sf*signal, histtype='step',color='midnightblue',label='signal')
# plt.hist(back_bins[:-1], back_bins, weights=back_sf*bkgrnd, histtype='step',color='firebrick',label='background')

# plt.yscale('log')
# plt.xlabel('Feature',fontsize=12)
# plt.ylabel('Events',fontsize=12)
# plt.legend(frameon=False)
# plt.show()

## Creating Datasets

In [ ]:
# data_high_glu = data[data.reco_g_1 > 0.7]
# data_high_glu = data[data.reco_g_2 > 0.7]

# Removing the gluon scores:
# no_score = data_high_glu.drop(columns=['reco_g_1', 'reco_g_2', 'reco_q_1', 'reco_q_2', 'reco_s_1', 'reco_s_2'])
# no_score

In [ ]:
# plt.figure()
# plt.hist(data_high_glu.reco_g_1[data_high_glu.label == 1],bins=np.linspace(0,1,50),
#          histtype='step',color='midnightblue',label='signal')
# plt.hist(data_high_glu.reco_g_1[data_high_glu.label == 0],bins=np.linspace(0,1,50),
#          histtype='step',color='firebrick',label='background')

# plt.xlabel('reco_glu_1',fontsize=12)
# plt.ylabel('Events',fontsize=12)
# plt.legend(frameon=False)

## Formatting the data for use with XGBoost

In [ ]:
shuf_data = data.sample(frac=1)

In [ ]:
shuf_data['label'] = data.label.astype('category')

Taking 80% of data for training set, 20% for test set.

In [ ]:
no_events = shuf_data.shape[0]
no_training = int(0.8 * no_events)

training_set = shuf_data[:no_training]
test_set = shuf_data[no_training:]

In [ ]:
print('Number of training samples: {}'.format(len(training_set)))
print('Number of testing samples: {}'.format(len(test_set)))

print('\nNumber of signal events in training set: {}'.format(len(training_set[training_set.label == 1])))
print('Number of background events in training set: {}'.format(len(training_set[training_set.label == 0])))
print('Fraction signal: {}'.format(len(training_set[training_set.label == 1])/(float)(len(training_set[training_set.label == 1]) + len(training_set[training_set.label == 0]))))

In [ ]:
# training_set.label.cat.codes
# print(training_set.label.cat.codes)

## Creating DMatrix

In [ ]:
feature_names = shuf_data.columns[0:-1]  # skip the final collumn as it is the label
# print(feature_names)
train = xgb.DMatrix(data=training_set[feature_names],label=training_set.label.cat.codes,
                    missing=-999.0,feature_names=feature_names)
test = xgb.DMatrix(data=test_set[feature_names],label=test_set.label.cat.codes,
                   missing=-999.0,feature_names=feature_names)

In [ ]:
print('Number of training samples: {}'.format(train.num_row()))
print('Number of testing samples: {}'.format(test.num_row()))

print('\nNumber of signal events in training set: {}'.format(len(np.where(train.get_label())[0])))

## Hyperparameters

## Evaluating Hyperparmameters

In [ ]:
eta_range = np.arange(0.1, 1.0, 0.1)
max_depth_range = np.arange(2, 9, 1)
min_child_range = np.arange(0, 5.1, 0.1)
subsample_range = np.arange(0.1, 1.1, 0.1)
colsample_range = np.arange(0.1, 1.1, 0.1)
lambda_range = np.arange(0, 10.25, 0.25)
gamma_range = np.arange(0, 5.1, 0.1)

rmse_outputs = np.zeros(len(gamma_range))

no_params = len(eta_range) + len(max_depth_range) + len(min_child_range) + len(subsample_range) + len(colsample_range) + len(lambda_range) + len(gamma_range)

print(no_params)

In [ ]:
# # regex pattern to find floats in evaluator string
# float_pattern = r'\d+\.\d+'

# for i in range(len(gamma_range)):
#     print(i)
#     param = {}

#     # Booster parameters
#     param['eta']              = 0.1 # learning rate
#     param['max_depth']        = 6  # maximum depth of a tree
#     param['subsample']        = 1 # fraction of events to train tree on
#     param['colsample_bytree'] = 0.8 # fraction of features to train tree on
#     param['gamma'] = gamma_range[i]  # Minimum loss reduction

#     # L2 regularization
#     param['lambda'] = 3

#     # Learning task parameters
#     param['objective']   = 'binary:logistic' # objective function
#     param['eval_metric'] = 'error'           # evaluation metric for cross validation
#     param = list(param.items()) + [('eval_metric', 'logloss')] + [('eval_metric', 'rmse')]

#     num_trees = 250  # number of trees to make

#     eval_booster = xgb.train(param, train, num_boost_round=num_trees)
#     eval_res = eval_booster.eval(evaluation)

#     result = float(re.findall(float_pattern, eval_res)[0])
#     rmse_outputs[i] = result

# print(rmse_outputs)
# os.system("say beep")

In [ ]:
# plt.plot(gamma_range, rmse_outputs, 'o-', color='midnightblue')
# plt.title('RMSE vs Gamma')
# plt.xlabel('Gamma')
# plt.ylabel('RMSE')

## Random Hyperparameter Search

In [ ]:
params = {
    'eta':eta_range,
    'max_depth':max_depth_range, 
    'gamma':gamma_range,  
    'subsample':subsample_range,
    'min_child_weight':min_child_range,
    'colsample_bytree':colsample_range, 
    'objective': ['binary:logistic'],
    'eval_metric': ['rmse'],
    'lambda':lambda_range,
}

In [ ]:
cla = XGBClassifier()

In [ ]:
# train test split with randomization performed (although randomization is not necessary)
hyperparam_training_set = shuf_data[:no_training].label.astype(int)
X_train, y_train  = training_set.iloc[:,:-1], training_set.label

In [ ]:
random_search = RandomizedSearchCV(cla, param_distributions=params, n_iter=30, cv=3, verbose=3)

In [ ]:
random_search.fit(X_train, y_train)
os.system("say bing bong")

In [ ]:
hyper_res = random_search.cv_results_
mean_score = hyper_res['mean_test_score']
err = hyper_res['std_test_score']
x_arr = np.arange(1, 31, 1)
# hyper_res

In [ ]:
plt.errorbar(x_arr, mean_score, yerr=err)
plt.xlabel('iteration')
plt.ylabel('score')

In [ ]:
hyper_params = random_search.best_params_
hyper_params

# Training

## Default Parameters

In [ ]:
param = {}

# Booster parameters
param['eta']              = 0.1 # learning rate
param['max_depth']        = 10  # maximum depth of a tree
param['subsample']        = 0.8 # fraction of events to train tree on
param['colsample_bytree'] = 0.8 # fraction of features to train tree on

# Learning task parameters
param['objective']   = 'binary:logistic' # objective function
param['eval_metric'] = 'error'           # evaluation metric for cross validation
param = list(param.items()) + [('eval_metric', 'logloss')] + [('eval_metric', 'rmse')]

num_trees = 100  # number of trees to make

## Parameters from random search

In [ ]:
# param = {}

# # Booster parameters
# param['eta']              = 0.1 # learning rate
# param['min_child_weight'] = 1.9000000000000001
# param['max_depth']        = 8  # maximum depth of a tree
# param['subsample']        = 0.4 # fraction of events to train tree on
# param['colsample_bytree'] = 0.8 # fraction of features to train tree on
# param['gamma'] = 4.00

# # L2 regularization
# param['lambda'] = 3.7

# # Learning task parameters
# param['objective']   = 'binary:logistic' # objective function
# param['eval_metric'] = 'error'           # evaluation metric for cross validation
# param = list(param.items()) + [('eval_metric', 'logloss')] + [('eval_metric', 'rmse')]

# num_trees = 600  # number of trees to make

In [ ]:
booster = xgb.train(param,train, num_boost_round=num_trees, evals=[(test, 'test')], early_stopping_rounds=10)
os.system("say beep")

In [ ]:
print(booster.eval(test))

In [ ]:
predictions = booster.predict(test)

In [ ]:
att = booster.attributes()
att

In [ ]:
# booster_df = booster.trees_to_dataframe()

In [ ]:
best_tree = booster.best_iteration

In [ ]:
print(best_tree)

In [ ]:
# selected_feature_df = booster_df[booster_df['Feature'] == 'jj_m']

In [ ]:
# selected_feature_df[selected_feature_df['Tree'] == 999]

In [ ]:
# plot all predictions (both signal and background)
plt.figure()
plt.hist(predictions,bins=np.linspace(0,1,50),histtype='step',color='darkgreen',label='All events')
# make the plot readable
plt.xlabel('Prediction from BDT',fontsize=12)
plt.ylabel('Events',fontsize=12)
plt.legend(frameon=False)

# plot signal and background separately
plt.figure()
plt.hist(predictions[test.get_label().astype(bool)],bins=np.linspace(0,1,50),
         histtype='step',color='midnightblue',label='signal')
plt.hist(predictions[~(test.get_label().astype(bool))],bins=np.linspace(0,1,50),
         histtype='step',color='firebrick',label='background')
# make the plot readable
plt.xlabel('Prediction from BDT',fontsize=12)
plt.ylabel('Events',fontsize=12)
plt.legend(frameon=False)

In [ ]:
signal, sig_bins = np.histogram(predictions[test.get_label().astype(bool)],bins=np.linspace(0,1,50))
bkgrnd, back_bins = np.histogram(predictions[~(test.get_label().astype(bool))],bins=np.linspace(0,1,50))

sig_sf = 10 * 7.38400e-05
back_sf = (10 * 363) 

plt.hist(sig_bins[:-1], sig_bins, weights=sig_sf*signal, histtype='step',color='midnightblue',label='signal')
plt.hist(back_bins[:-1], back_bins, weights=back_sf*bkgrnd, histtype='step',color='firebrick',label='background')

plt.yscale('log')
plt.xlabel('Prediction from BDT',fontsize=12)
plt.ylabel('Events',fontsize=12)
plt.legend(frameon=False)
plt.show()

In [ ]:
# choose score cuts:
cuts = np.linspace(0,1,500)
nsignal = np.zeros(len(cuts))
nbackground = np.zeros(len(cuts))
for i,cut in enumerate(cuts):
    nsignal[i] = len(np.where(predictions[test.get_label().astype(bool)] > cut)[0])
    nbackground[i] = len(np.where(predictions[~(test.get_label().astype(bool))] > cut)[0])

# plot efficiency vs. purity (ROC curve)
plt.figure()
plt.plot(nsignal/len(test_set[test_set.label == 1]),nsignal/(nsignal + nbackground),'o-',color='blueviolet')
# make the plot readable
plt.xlabel('Efficiency',fontsize=12)
plt.ylabel('Purity',fontsize=12)

In [ ]:
# Zoom in view of the upper left corner.
plt.figure()
plt.xlim(0.85, 1.0)
plt.ylim(0.85, 1.0)
# plt.plot([0, 1], [0, 1], 'k--')
plt.plot(nsignal/len(test_set[test_set.label == 1]),nsignal/(nsignal + nbackground),'o-',color='blueviolet')
plt.xlabel('Efficiency',fontsize=12)
plt.ylabel('Purity',fontsize=12)
plt.title('ROC curve (zoomed in at top right)')
plt.show()

In [ ]:
xgb.plot_importance(booster,grid=False)

In [ ]:
plt.figure()
plt.hist(training_set.jj_eta_1[training_set.label == 1],bins=np.linspace(-4,4,50),
         histtype='step',color='midnightblue',label='signal')
plt.hist(training_set.jj_eta_1[training_set.label == 0],bins=np.linspace(-4,4,50),
         histtype='step',color='firebrick',label='background')

plt.xlabel('jj_p_1',fontsize=12)
plt.ylabel('Events',fontsize=12)
plt.legend(frameon=False)

In [ ]:
plt.figure()
plt.plot(training_set.jj_p_1[training_set.label == 1],training_set.jj_e_1[training_set.label == 1],
         'o',markersize=1.5,color='mediumblue',markeredgewidth=0,alpha=0.7,label='signal')
plt.plot(training_set.jj_p_1[training_set.label == 0],training_set.jj_e_1[training_set.label == 0],
         'o',markersize=1.5,color='firebrick',markeredgewidth=0,alpha=0.8,label='background')

plt.xlim(10,80)
plt.ylim(20,100)
plt.xlabel('reco_s_1',fontsize=12)
plt.ylabel('reco_s_2',fontsize=12)
plt.legend(frameon=False,numpoints=1,markerscale=2)

In [ ]:
plt.figure()
plt.plot(training_set.reco_g_1[training_set.label == 1],training_set.reco_g_2[training_set.label == 1],
         'o',markersize=1,color='mediumblue',markeredgewidth=0,alpha=0.8,label='signal')
plt.plot(training_set.reco_g_1[training_set.label == 0],training_set.reco_g_2[training_set.label == 0],
         'o',markersize=2,color='firebrick',markeredgewidth=0,alpha=1,label='background')

plt.xlim(0.7,1)
plt.ylim(0.7,1)
plt.xlabel('reco_g_1',fontsize=12)
plt.ylabel('reco_g_2',fontsize=12)
plt.legend(frameon=False,numpoints=1,markerscale=1)

In [ ]:
plt.figure()
plt.plot(training_set.jj_m[training_set.label == 1],training_set.jj_p_2[training_set.label == 1],
         'o',markersize=2,color='mediumblue',markeredgewidth=0,alpha=0.8,label='signal')
plt.plot(training_set.jj_m[training_set.label == 0],training_set.jj_p_2[training_set.label == 0],
         'o',markersize=2,color='firebrick',markeredgewidth=0,alpha=0.8,label='background')

plt.xlim(0, 140)
plt.ylim(0, 75)
plt.xlabel('jj_m',fontsize=12)
plt.ylabel('jj_p_2',fontsize=12)
plt.legend(frameon=False,numpoints=1,markerscale=1)